In [64]:
import openai
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

openai.api_key = "sk-or-vv-79c5f99f4e7122fe2efec39a789e0c4aef07389d44052275c9fda0b3918c1857"
openai.base_url = "https://api.vsegpt.ru/v1/"

system_text = "Ты генератор отзывов о вебинарах, курсах."

arr1 = ["к вебинару", "об преподавателе с вебинара", "к программе вебинара"]
arr2 = ["положительный", "отрицательный"]
df = pd.DataFrame()
for i in range(50):
    for j in range(len(arr1)):
        for k in range(len(arr2)):
            promt = f"Сгененируй краткий {arr2[k]} отзыв {arr1[j]} по программированию"
            jjh = '11233'  # это типо переменная для выхлопа нейронки
            data = pd.DataFrame([{'feed': jjh, 'for what': arr1[j]}])
            df = pd.concat([df, data])

def get_resp(messages):
    response_big = openai.ChatCompletion.create(

        model="openai/gpt-3.5-turbo",
        messages=messages,
        temperature=0.9,
        n=1,
        max_tokens = 800,
        stream=False,
        tools = [{
            "type": "function",
            "function": {
                "name": "get_review",
                "description": "Get the review about courses",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "review_prepod": {
                            "type": "string",
                            "description": "Review about teacher, lecturer",
                        },
                        "review_vebinar": {"type": "string",
                                 "desciption": "Review about vebinar or lection"},
                        "review_program": {"type": "string",
                                          "description": "Review about program of course"}
                    },
                    "required": ["review_prepod", "review_vebinar", "review_program"],
                },
            },
        }
        ]
    )
    response = response_big.choices[0].message.content
    return response

objects = {
    'к вебинару': 0,
    'об преподавателе с вебинара': 2,
    'к программе вебинара': 1
}

train_df = pd.DataFrame(columns=['text', 'label'])

# Use ThreadPoolExecutor to speed up processing
with ThreadPoolExecutor() as executor:
    for row in tqdm(df.to_dict('records')):
        # Process each row
        result = executor.submit(process_row, row).result()
        # Append result to train_df
        train_df = train_df.append(result, ignore_index=True)
        # Add a delay of 1.01 seconds between each request
        time.sleep(0.5)

# Append results to train_df
train_df = train_df.append(results, ignore_index=True)

  0%|          | 0/300 [00:00<?, ?it/s]


APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [65]:
train_df

,text,label


In [62]:
train_df.to_csv("C:/Users/NightMare/Desktop/neurofeed_back/Preprocessdata/train_df_obj4.csv", index=False)